In [ ]:
from pathlib import Path
import sys

import pandas as pd
import numpy as np

# repo layout: compute repo/backends relative to cwd so notebook is portable
REPO_ROOT = Path.cwd().resolve().parents[3]
BACKEND_ROOT = REPO_ROOT / "backend"
DATA_DIR = BACKEND_ROOT / "app" / "data" / "impact_training_data"

# load the four input CSVs; use str(Path) to avoid pandas filepath issues
re_ee = pd.read_csv(str(DATA_DIR / "re_ee.csv"))
transport = pd.read_csv(str(DATA_DIR / "transport.csv"))
water = pd.read_csv(str(DATA_DIR / "water_urban_infra.csv"))
blue = pd.read_csv(str(DATA_DIR / "blue.csv"))

# quick shape check to confirm files loaded as expected
for name, df in [("re_ee", re_ee), ("transport", transport),
                ("water", water), ("blue", blue)]:
    print(f"{name}: {df.shape[0]} rows, {df.shape[1]} columns")

re_ee: 83 rows, 16 columns
transport: 64 rows, 9 columns
water: 19 rows, 9 columns
blue: 19 rows, 10 columns


In [ ]:
# Cell 2: helper functions

import re

def find_target_results_column(df: pd.DataFrame) -> str | None:
    """
    For transport/water/blue: try to find a 'Target Results' style column.
    """
    # search column headers for a likely target-results field
    for col in df.columns:
        if "target result" in col.lower():
            return col
    return None


def extract_co2_from_text(s: str):
    """
    Extract a numeric CO2 value in tons from free text like:
    'Avoids 15,000 tCO2e per year' or '10,500 tons of CO2 avoided'
    Returns float or None.
    """
    if not isinstance(s, str):
        return None

    # normalize to lowercase for simple regex matching
    text = s.lower()

    # look for patterns like '15,000', '10.5', etc. near 't' or 'ton'
    # this is a lightweight heuristic; it will miss many complex formats
    match = re.search(r"([\d.,]+)\s*(?:t|ton)", text)
    if not match:
        return None

    # remove thousands separators and convert to float
    num_str = match.group(1).replace(",", "")
    try:
        return float(num_str)
    except ValueError:
        return None


In [ ]:
# Cell 3: process RE/EE data (has explicit numeric label)

# try to detect a human-readable project/description column if helper exists
try:
    re_ee_desc_col = find_description_column(re_ee)
except NameError:
    re_ee_desc_col = None

# Known label names we want to map to normalized names
re_ee_label_col = 'Annual GHG Emission Avoided (ton of CO2 Equivalent)b'
proj_candidate = None
# try to find a project/description-like column robustly using header heuristics
for c in re_ee.columns:
    cl = c.lower()
    if ('project name' in cl and 'description' in cl) or ('project' in cl and 'description' in cl):
        proj_candidate = c
        break
# fallback: match a known header string present in some source files
if proj_candidate is None and 'Project Name \n+(Number/Year Loan Approved) \nand Description'.lower() in '\n'.join(re_ee.columns).lower():
    proj_candidate = 'Project Name \n+(Number/Year Loan Approved) \nand Description'

# build list of columns to keep (only include those actually present)
keep_cols = []
if proj_candidate and proj_candidate in re_ee.columns:
    keep_cols.append(proj_candidate)
if re_ee_label_col in re_ee.columns:
    keep_cols.append(re_ee_label_col)

# include optional contextual columns if present (focus area, total cost, category)
focus_col = None
for c in re_ee.columns:
    if 'ocean finance framework - primary focus area' in c.lower():
        focus_col = c
        keep_cols.append(c)
# note: some CSVs store the total cost header with a newline
total_cost_col = 'Total Project Cost\n($ million)'
if total_cost_col in re_ee.columns:
    keep_cols.append(total_cost_col)
re_or_ee_col = 'RE or EE'
if re_or_ee_col in re_ee.columns:
    keep_cols.append(re_or_ee_col)

if not keep_cols:
    # fail fast if no expected columns exist in this source
    raise ValueError('No expected columns found in re_ee to build output dataframe')

# create new dataframe with only the kept columns and rename them to safe names
col_renames = {}
if proj_candidate:
    col_renames[proj_candidate] = 'project_description'
if re_ee_label_col in re_ee.columns:
    col_renames[re_ee_label_col] = 'actual_co2_tons'
if focus_col:
    col_renames[focus_col] = 'primary_focus_area'
# rename the total cost column to a safer numeric name
col_renames[total_cost_col] = 'total_project_cost_mil_usd'
# keep the RE/EE marker as 'category'
if re_or_ee_col in re_ee.columns:
    col_renames[re_or_ee_col] = 'category'

re_ee_clean = re_ee[keep_cols].rename(columns=col_renames).copy()

# clean the actual_co2_tons column: strip non-numeric chars and coerce to numeric
if 'actual_co2_tons' in re_ee_clean.columns:
    re_ee_clean['actual_co2_tons'] = re_ee_clean['actual_co2_tons'].astype(str).str.replace('[^0-9.\-]', '', regex=True)
    re_ee_clean['actual_co2_tons'] = pd.to_numeric(re_ee_clean['actual_co2_tons'], errors='coerce')
    # drop rows missing numeric CO2 values (optional for downstream training)
    re_ee_clean = re_ee_clean[re_ee_clean['actual_co2_tons'].notna()].copy()
    # convert to pandas nullable integer type for compact storage
    re_ee_clean['actual_co2_tons'] = re_ee_clean['actual_co2_tons'].astype('Int64')

# show resulting columns and frame
print('Output columns:', list(re_ee_clean.columns))
re_ee_clean

Output columns: ['project_description', 'actual_co2_tons', 'total_project_cost_mil_usd', 'category']


,project_description,actual_co2_tons,total_project_cost_mil_usd,category
0,Indonesia: Java–Bali Electricity Distribution ...,330000,120.00,EE
1,"China, People's Republic of: Integrated Renewa...",1000000,152.54,RE
2,Papua New Guinea: Town Electrification Investm...,35000,71.60,RE
4,"China, People’s Republic of:Agricultural and\n...",255200,200.00,RE
5,Regional: Southeast Asia Energy Efficiency Pro...,90000,200.00,EE
...,...,...,...,...
78,Bhutan: Distributed Solar for Public Infrastru...,39735,34.00,RE
79,Solomon Islands: Renewable Energy Development ...,5600,42.00,RE
80,Azerbaijan: Bilasuvar Solar Power Project (453...,426152,not disclosedh,RE
81,Azerbaijan: Banka Solar Power Project (4536/FY...,302972,not disclosedh,RE


In [ ]:
# Cell 4: process transport / water / blue via text extraction

def process_text_dataset(df: pd.DataFrame, name: str) -> pd.DataFrame:
    """Try to detect a description/project column and extract CO2 from a target-results column.

    Detection strategy (in order):
    - column containing both 'project' and 'description'
    - column containing 'project' and 'name' or 'title'
    - any column containing 'description'
    - fallback to first object-typed column
    """
    # detect description column using heuristics over header names
    desc_col = None
    cols = list(df.columns)
    for c in cols:
        lc = c.lower()
        if 'project' in lc and 'description' in lc:
            desc_col = c
            break
    if desc_col is None:
        for c in cols:
            lc = c.lower()
            if 'project' in lc and ('name' in lc or 'title' in lc):
                desc_col = c
                break
    if desc_col is None:
        for c in cols:
            if 'description' in c.lower():
                desc_col = c
                break
    if desc_col is None:
        # fallback: first object/string column
        obj_cols = [c for c in cols if df[c].dtype == 'object']
        if obj_cols:
            desc_col = obj_cols[0]

    if desc_col is None:
        print(f"[{name}] Could not detect a description/project column. Available columns:\n  {cols}")
        return pd.DataFrame(columns=["project_description", "actual_co2_tons"])

    # find the column that likely contains target/result strings to extract numeric co2
    target_col = find_target_results_column(df)
    if target_col is None:
        print(f"[{name}] No 'Target Results' style column found. Skipping labels.")
        return pd.DataFrame(columns=["project_description", "actual_co2_tons"])

    # select columns to keep; include total cost if present
    cols_to_use = [desc_col, target_col]
    total_cost_col = 'Total Project Cost\n($ million)'
    if total_cost_col in df.columns:
        cols_to_use.append(total_cost_col)

    if desc_col not in df.columns or target_col not in df.columns:
        print(f"[{name}] Required columns not found: desc={desc_col in df.columns}, target={target_col in df.columns}")
        return pd.DataFrame(columns=["project_description", "actual_co2_tons"])

    temp = df[cols_to_use].copy()
    temp = temp.rename(columns={desc_col: "project_description"})

    # normalize cost header to safe numeric name if present
    if total_cost_col in temp.columns:
        temp = temp.rename(columns={total_cost_col: 'total_project_cost_mil_usd'})

    # extract numeric co2 values from the target/results text column
    temp["actual_co2_tons"] = temp[target_col].apply(extract_co2_from_text)
    before = temp.shape[0]
    temp = temp.dropna(subset=["actual_co2_tons"])
    after = temp.shape[0]

    # tag transport rows explicitly for downstream use
    if name.lower() == 'transport':
        temp['category'] = 'transport'

    print(f"[{name}] kept {after}/{before} rows with numeric CO2 extracted.")
    # prepare output columns (include cost/category when present)
    cols_out = ['project_description', 'actual_co2_tons']
    if 'total_project_cost_mil_usd' in temp.columns:
        cols_out.append('total_project_cost_mil_usd')
    if 'category' in temp.columns:
        cols_out.append('category')

    return temp[cols_out]


transport_clean = process_text_dataset(transport, "transport")

transport_clean

[transport] kept 19/64 rows with numeric CO2 extracted.


,project_description,actual_co2_tons,total_project_cost_mil_usd,category
0,Turkmenistan: North–South Railway (2737/FY2011...,26800.000,166.70,transport
4,Bangladesh: Greater Dhaka Sustainable Urban Tr...,40000.000,255.00,transport
7,"China, People's Republic of: Railway Energy Ef...",2008.000,547.60,transport
9,"China, People's Republic of: Railway Energy Ef...",2009.000,580.15,transport
10,Bangladesh: SASEC Railway Connectivity—Akhaura...,14.000,805.00,transport
20,"Thailand: Bangkok Mass Rapid Transit (3669, 36...",50000.000,"2,960.00",transport
22,Uzbekistan: Railway Efficiency Improvement Pro...,900000.000,218.30,transport
26,Pakistan: Karachi Bus Rapid Transit Red Line P...,77979.000,323.00,transport
34,"China, People's Republic of: Jilin Yanji Low-C...",60000.000,259.79,transport
38,India: Delhi–Meerut Regional Rapid Transit Sys...,258035.000,"3,949.70",transport


In [ ]:
# Cell 5: combine all datasets, standardize names, add log transform, and save to CSV

# normalize textual project column across frames to 'project_name' if present
for df_name in ['transport_clean', 'water_clean', 'blue_clean']:
    if df_name in globals():
        df = globals()[df_name]
        if 'project_description' in df.columns:
            globals()[df_name] = df.rename(columns={'project_description': 'project_name'})

# ensure we created the re_ee_clean frame earlier
if 're_ee_clean' not in globals():
    raise RuntimeError('Expected re_ee_clean to be present from prior cell')

# collect available cleaned frames into a list for concatenation
dfs = []
for name in ['re_ee_clean', 'transport_clean', 'water_clean', 'blue_clean']:
    if name in globals():
        dfs.append(globals()[name])

# choose a common set of columns to keep when present
common_cols = ['project_name', 'actual_co2_tons', 'primary_focus_area', 'total_project_cost_mil_usd', 'category']
dfs_selected = []
for df in dfs:
    # repair older total cost header if present
    old_total_col = 'Total Project Cost\n($ million)'
    if old_total_col in df.columns:
        df = df.rename(columns={old_total_col: 'total_project_cost_mil_usd'})
    available = [c for c in common_cols if c in df.columns]
    # defensive: map project_description -> project_name when needed
    if 'project_name' not in available and 'project_description' in df.columns:
        df = df.rename(columns={'project_description': 'project_name'})
        available = [c for c in common_cols if c in df.columns]
    dfs_selected.append(df[available].copy())

# concatenate rows from all sources into a single dataset
all_data = pd.concat(dfs_selected, ignore_index=True, sort=False)

# coerce CO2 to numeric and drop rows without numeric values (required for training)
if 'actual_co2_tons' in all_data.columns:
    all_data['actual_co2_tons'] = pd.to_numeric(all_data['actual_co2_tons'], errors='coerce')
    before = all_data.shape[0]
    all_data = all_data[all_data['actual_co2_tons'].notna()].copy()
    after = all_data.shape[0]
    print(f'Dropped {before-after} rows without numeric actual_co2_tons')
    # convert to pandas nullable integer type for compactness
    all_data['actual_co2_tons'] = all_data['actual_co2_tons'].astype('Int64')
else:
    raise RuntimeError('No actual_co2_tons column present after concatenation')

# add a log-transformed target column (log1p to handle zeroes)
all_data['log_co2'] = np.log1p(all_data['actual_co2_tons'].astype(float))

print('all_data shape:', all_data.shape)
print(all_data['actual_co2_tons'].describe())

# persist consolidated training CSV into backend data folder
out_path = BACKEND_ROOT / 'app' / 'data' / 'impact_training_dataset.csv'
all_data.to_csv(str(out_path), index=False)
print('Saved consolidated CSV to', out_path)

all_data.head()

Dropped 0 rows without numeric actual_co2_tons
all_data shape: (96, 5)
count             96.0
mean     346930.302083
std      807911.585956
min                0.0
25%            29570.0
50%            88984.0
75%           381250.0
max          7000000.0
Name: actual_co2_tons, dtype: Float64
Saved consolidated CSV to /Users/radhamunver/Downloads/GreenPrism/green-prism/backend/app/data/impact_training_all.csv


,project_name,actual_co2_tons,total_project_cost_mil_usd,category,log_co2
0,Indonesia: Java–Bali Electricity Distribution ...,330000,120.00,EE,12.706851
1,"China, People's Republic of: Integrated Renewa...",1000000,152.54,RE,13.815512
2,Papua New Guinea: Town Electrification Investm...,35000,71.60,RE,10.463132
3,"China, People’s Republic of:Agricultural and\n...",255200,200.00,RE,12.449807
4,Regional: Southeast Asia Energy Efficiency Pro...,90000,200.00,EE,11.407576


In [6]:
all_data

,project_name,actual_co2_tons,total_project_cost_mil_usd,category,log_co2
0,Indonesia: Java–Bali Electricity Distribution ...,330000,120.00,EE,12.706851
1,"China, People's Republic of: Integrated Renewa...",1000000,152.54,RE,13.815512
2,Papua New Guinea: Town Electrification Investm...,35000,71.60,RE,10.463132
3,"China, People’s Republic of:Agricultural and\n...",255200,200.00,RE,12.449807
4,Regional: Southeast Asia Energy Efficiency Pro...,90000,200.00,EE,11.407576
...,...,...,...,...,...
91,India: Nhavae Sheva Container Terminal Financi...,4977,187.10,transport,8.512783
92,Bangladesh: Flood Reconstruction Emergency Ass...,1,289.17,transport,0.693147
93,Georgia: Tegeta Green Vehicles Bond Project (7...,1636,7.69,transport,7.400621
94,Philippines: Davao Public Transport Modernizat...,3,"1,710.78",transport,1.386294
